In [24]:
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer

In [5]:
data = pd.read_csv('train_data.csv', index_col=0)
data.head()

,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit
ID,,,,,,,,,
0,J06.0,Острый ларингофарингит,29,2,NaN,NaN,NaN,На жевательной поверхности 2.6- кариозная поло...,1
1,N76.1,Подострый и хронический вагинит,45,2,NaN,NaN,считает себя больной на протяжении многих лет....,NaN,1
2,L23.9,"Аллергический контактный дерматит, причина не ...",63,2,NaN,NaN,"Болен около 2-х дней, когда появились описанны...",NaN,1
3,N48.1,Баланопостит,43,2,NaN,не отягощен,NaN,NaN,1
4,Z00.0,Общий медицинский осмотр,29,1,NaN,NaN,провела полное обследование для оперативного л...,NaN,1


In [64]:
str_anamnez = data[~pd.isnull(data.iloc[:, 6])].iloc[:, 6]
stemmer = RussianStemmer()
analyzer = CountVectorizer().build_analyzer()

In [66]:
def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

In [67]:
vec = CountVectorizer(tokenizer=stemmed_words).fit(str_anamnez)
bag_of_words = vec.transform(str_anamnez)
sum_words = bag_of_words.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
words_stop = filter(lambda x: x[0] not in stopwords.words('russian'), words_freq)
sorted_bow = sorted(words_stop, key = lambda x: x[1], reverse=True)

In [68]:
sorted_bow

[('бол', 68806),
 ('лечен', 52653),
 ('жалоб', 46727),
 ('течен', 45896),
 ('беспоко', 36089),
 ('обследован', 35720),
 ('отмеча', 34476),
 ('недел', 32196),
 ('лет', 31300),
 ('окол', 30912),
 ('фон', 30762),
 ('обрат', 29432),
 ('принима', 28918),
 ('назад', 26899),
 ('год', 25968),
 ('месяц', 25665),
 ('посл', 25502),
 ('леч', 23823),
 ('последн', 23777),
 ('самостоятельн', 23741),
 ('дан', 21650),
 ('появ', 21524),
 ('дне', 21278),
 ('терап', 20612),
 ('связыва', 19587),
 ('консультац', 18257),
 ('результат', 17999),
 ('динамик', 17526),
 ('10', 17017),
 ('эффект', 15984),
 ('обраща', 15914),
 ('положительн', 15488),
 ('слов', 15427),
 ('ухудшен', 14124),
 ('ран', 14019),
 ('заболеван', 13915),
 ('прав', 13779),
 ('анамнез', 13571),
 ('больн', 13536),
 ('обострен', 13299),
 ('лев', 13137),
 ('проведен', 12621),
 ('травм', 12361),
 ('пациентк', 12309),
 ('ст', 12191),
 ('нескольк', 11970),
 ('повод', 11959),
 ('нача', 11731),
 ('улучшен', 11719),
 ('врач', 11502),
 ('врем', 11452),


In [69]:
words = sorted_bow[:300]
words

[('бол', 68806),
 ('лечен', 52653),
 ('жалоб', 46727),
 ('течен', 45896),
 ('беспоко', 36089),
 ('обследован', 35720),
 ('отмеча', 34476),
 ('недел', 32196),
 ('лет', 31300),
 ('окол', 30912),
 ('фон', 30762),
 ('обрат', 29432),
 ('принима', 28918),
 ('назад', 26899),
 ('год', 25968),
 ('месяц', 25665),
 ('посл', 25502),
 ('леч', 23823),
 ('последн', 23777),
 ('самостоятельн', 23741),
 ('дан', 21650),
 ('появ', 21524),
 ('дне', 21278),
 ('терап', 20612),
 ('связыва', 19587),
 ('консультац', 18257),
 ('результат', 17999),
 ('динамик', 17526),
 ('10', 17017),
 ('эффект', 15984),
 ('обраща', 15914),
 ('положительн', 15488),
 ('слов', 15427),
 ('ухудшен', 14124),
 ('ран', 14019),
 ('заболеван', 13915),
 ('прав', 13779),
 ('анамнез', 13571),
 ('больн', 13536),
 ('обострен', 13299),
 ('лев', 13137),
 ('проведен', 12621),
 ('травм', 12361),
 ('пациентк', 12309),
 ('ст', 12191),
 ('нескольк', 11970),
 ('повод', 11959),
 ('нача', 11731),
 ('улучшен', 11719),
 ('врач', 11502),
 ('врем', 11452),


In [97]:
def change_dataset(words, data, col_num):
    new_data = pd.DataFrame(columns=words, index=data.index)
    for ind, row in new_data.iterrows():
        if pd.isnull(data.iloc[ind, col_num]):
            continue
        for word in words:
#             print((r'\b' + word, data.iloc[ind, col_num]))
            new_data.loc[ind, word] = len(re.findall(r'\b' + word, data.iloc[ind, col_num]))
    return pd.concat([new_data, data], axis=1)

In [95]:
clear_words = list(map(lambda pair: pair[0], words))
clear_words

['бол',
 'лечен',
 'жалоб',
 'течен',
 'беспоко',
 'обследован',
 'отмеча',
 'недел',
 'лет',
 'окол',
 'фон',
 'обрат',
 'принима',
 'назад',
 'год',
 'месяц',
 'посл',
 'леч',
 'последн',
 'самостоятельн',
 'дан',
 'появ',
 'дне',
 'терап',
 'связыва',
 'консультац',
 'результат',
 'динамик',
 '10',
 'эффект',
 'обраща',
 'положительн',
 'слов',
 'ухудшен',
 'ран',
 'заболеван',
 'прав',
 'анамнез',
 'больн',
 'обострен',
 'лев',
 'проведен',
 'травм',
 'пациентк',
 'ст',
 'нескольк',
 'повод',
 'нача',
 'улучшен',
 'врач',
 'врем',
 'сустав',
 'отдел',
 '17',
 'мм',
 'остр',
 'вышеуказа',
 'диагноз',
 'головн',
 'когд',
 'длительн',
 'курс',
 'настоя',
 'ад',
 'позвоночник',
 'проход',
 'област',
 'состоян',
 'обследова',
 'дня',
 'себ',
 '16',
 'пациент',
 'тел',
 '12',
 'рт',
 'счита',
 'нагрузк',
 'невролог',
 'осмотр',
 'повторн',
 'нос',
 'стал',
 'появлен',
 'орв',
 'мрт',
 'периодическ',
 '15',
 'горл',
 'сохраня',
 '11',
 'повышен',
 '2016',
 'вышеописа',
 'провод',
 'ден',


In [ ]:
import re
new_dataset = change_dataset(clear_words, data, 6)

In [ ]:
new_dataset.head()